# 01.半结构化数据上的RAG
- 典型的半结构化数据是包含文本、表格、图像等的PDF文档。



In [2]:
# 下载PDF文件并将其命名为 statement_of_changes.pdf 。
!wget -O statement_of_changes.pdf https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/381953f9-934e-4cc8-b099-144910676bad.pdf


--2024-06-28 15:02:05--  https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/381953f9-934e-4cc8-b099-144910676bad.pdf
Resolving d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)... 18.155.204.129, 18.155.204.91, 18.155.204.120, ...
Connecting to d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)|18.155.204.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119037 (116K) [application/pdf]
Saving to: ‘statement_of_changes.pdf’

statement_of_change 100%[===================>] 116.25K   356KB/s    in 0.3s    

2024-06-28 15:02:06 (356 KB/s) - ‘statement_of_changes.pdf’ saved [119037/119037]



In [7]:
# apt-get 是安装系统级的软件包。
# !apt-get install poppler-utils tesseract-ocr
# !brew install poppler-utils tesseract-ocr
# !brew install poppler-qt5
# !brew install poppler-utils

==> Searching for similarly named formulae and casks...
==> Formulae
poppler-qt5

To install poppler-qt5, run:
  brew install poppler-qt5


In [6]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

raw_pdf_elements = partition_pdf(
    filename="statement_of_changes.pdf",
    extract_images_in_pdf=False,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=".",
)

ValueError: unstructured_inference is not installed, pytesseract is not installed and the text of the PDF is not extractable. To process this file, install unstructured_inference, install pytesseract, or remove copy protection from the PDF.

In [ ]:

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt_text = """
  You are responsible for concisely summarizing table or text chunk:

  {element}
"""
prompt = ChatPromptTemplate.from_template(prompt_text)
summarize_chain = ({"element": lambda x: x}
                   | prompt
                   | ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
                   | StrOutputParser())


In [ ]:
import uuid

from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma

id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings()),
    docstore=InMemoryStore(),
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))
  